### 1 - PREPARAÇÃO E CARREGAMENTO DOS DADOS ###

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [2]:
try:
    df = pd.read_csv("flight_data_2024.csv")
    print("Arquivo carregado com sucesso!")

except FileNotFoundError:
    print("ERRO: Arquivo não encontrado")

print(f"Dataset carregado com {df.shape[0]} linhas e {df.shape[1]} colunas.")

Arquivo carregado com sucesso!
Dataset carregado com 1048575 linhas e 18 colunas.


In [3]:
try:
    df['flight_id'] = np.arange(df.shape[0]) + 1
    colunas = ['flight_id'] + [col for col in df.columns if col != 'flight_id']
    df = df[colunas]

    print(f"\n Coluna 'flight_id' criada com sucesso!")
    print(f"Total de {df.shape[0]:,} voos.")

    print("\n ---- 5 primeiras linhas do df com o ID novo ----")
    print(df[['flight_id', 'fl_date', 'origin', 'cancelled']].head(5))

except Exception as e:
    print(f"ocorreu um erro inesperado: {e}")


 Coluna 'flight_id' criada com sucesso!
Total de 1,048,575 voos.

 ---- 5 primeiras linhas do df com o ID novo ----
   flight_id   fl_date origin  cancelled
0          1  1/1/2024    JFK          0
1          2  1/1/2024    MSP          0
2          3  1/1/2024    JFK          0
3          4  1/1/2024    RIC          0
4          5  1/1/2024    DTW          0


In [4]:
coluna_alvo = 'cancelled'

df[coluna_alvo]=pd.to_numeric(df[coluna_alvo], errors='coerce').fillna(0).astype(int)

# c) Preparação de Variáveis Numéricas (taxiamento)
df['taxi_out'] = pd.to_numeric(df['taxi_out'], errors='coerce').fillna(0)

# d) Criação de Features de Tempo (data)
df['fl_date'] = pd.to_datetime(df['fl_date'], errors='coerce')
df['day_of_year'] = df['fl_date'].dt.dayofyear
df['week_of_year'] = df['fl_date'].dt.isocalendar().week.astype(int)

# e) Codificação de Variáveis Categóricas (One-Hot Encoding)
colunas_para_dummies = ['origin', 'origin_state_nm']
df_para_regressao_logistica = pd.get_dummies(df, columns=colunas_para_dummies, drop_first=True)

# f) Remoção de colunas que não podem ser Features
colunas_para_remover = [
        'fl_date',
        'origin_city_name',
        'flight_id' 
    ]
df_para_regressao_logistica.drop(columns=colunas_para_remover, inplace=True, errors='ignore')

# 2. Definição de X e Y

Y = df_para_regressao_logistica[coluna_alvo]
X = df_para_regressao_logistica.drop(columns=[coluna_alvo])

# Tratamento final de X: Remover colunas com NaN ou Infinitos
X = X.replace([np.inf, -np.inf], np.nan).dropna(axis=1)

print(f"Dataset pronto. X: {X.shape[0]} linhas, {X.shape[1]} features.")

# 3. Divisão Treino e Teste (usando stratify para manter a proporção de cancelamentos)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=1, stratify=Y)

print("\n--- Treinando Modelo de Regressão Logística ---")

# 4. Treinamento do Modelo
model = LogisticRegression(solver='liblinear', random_state=1, max_iter=1000)
model.fit(X_train, Y_train)

# 5. Previsão e Avaliação
Y_pred = model.predict(X_test)

print("\n=======================================================")
print("✅ RESULTADOS DA REGRESSÃO LOGÍSTICA (PREVISÃO DE CANCELAMENTO)")
print("=======================================================")

# Métricas de Desempenho
print(f"Acurácia: {accuracy_score(Y_test, Y_pred):.4f}")
print("\nRelatório de Classificação (Precision, Recall, F1-Score):")
# zero_division=0 evita o warning se uma classe não for prevista
print(classification_report(Y_test, Y_pred, zero_division=0))
print("\nMatriz de Confusão (real vs. previsto):")

cm = confusion_matrix(Y_test, Y_pred)

# 2. Configuração da Visualização
plt.figure(figsize=(6, 5))

# Criar o Heatmap (Mapa de Calor)
sns.heatmap(
    cm,
    annot=True,              # Mostrar os números dentro dos quadrados
    fmt='d',                 # Formatar os números como inteiros
    cmap='Blues',            # Escolher um esquema de cores (Azul é um bom padrão)
    linewidths=0.5,          # Linhas entre os quadrados
    linecolor='black',
    cbar=False               # Não mostrar a barra de cores lateral
)

# 3. Adicionar Rótulos e Títulos
# Os rótulos 0 e 1 correspondem a 'Não Cancelado' e 'Cancelado'
class_names = ['Não Cancelado (0)', 'Cancelado (1)']
plt.xticks(np.arange(len(class_names)) + 0.5, class_names, rotation=45, ha='right')
plt.yticks(np.arange(len(class_names)) + 0.5, class_names, rotation=0)

plt.title('Matriz de Confusão (Previsão de Cancelamento)', fontsize=14)
plt.xlabel('Valor Previsto')
plt.ylabel('Valor Real')

plt.tight_layout()
plt.show()

print("\n*Lembre-se: Para a Classe 1 (Cancelados), o Recall é geralmente a métrica mais importante!*")



Dataset pronto. X: 1048575 linhas, 394 features.

--- Treinando Modelo de Regressão Logística ---


MemoryError: Unable to allocate 2.15 GiB for an array with shape (734002, 394) and data type float64

In [ ]:
# 3. Divisão Treino e Teste (usando 70/30 e random_state para consistência)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=1, stratify=Y)

print("\n--- Treinando Modelo de Árvore de Decisão ---")

    # 4. Treinamento do Modelo (Parâmetros iniciais: max_depth limita o crescimento da árvore)
model_tree = DecisionTreeClassifier(
        max_depth=5,        # Limita a profundidade para evitar Overfitting e facilitar a inspeção
        random_state=1,
        class_weight='balanced' # Tenta melhorar o Recall da Classe 1 (Cancelado)
    )
model_tree.fit(X_train, Y_train)

    # 5. Previsão e Avaliação
Y_pred_tree = model_tree.predict(X_test)

print("\n=======================================================")
print("✅ RESULTADOS DA ÁRVORE DE DECISÃO (PREVISÃO DE CANCELAMENTO)")
print("=======================================================")

    # Métricas de Desempenho
print(f"Acurácia: {accuracy_score(Y_test, Y_pred_tree):.4f}")
print("\nRelatório de Classificação (Precision, Recall, F1-Score):")
print(classification_report(Y_test, Y_pred_tree, zero_division=0))
print("\nMatriz de Confusão (real vs. previsto):")

cm_tree = confusion_matrix(Y_test, Y_pred_tree)
# 2. Configuração da Visualização
plt.figure(figsize=(6, 5))

# Criar o Heatmap (Mapa de Calor)
sns.heatmap(
    cm_tree,
    annot=True,              
    fmt='d',                 
    cmap='Greens',            
    linewidths=0.5,
    linecolor='black',
    cbar=False
)

# 3. Adicionar Rótulos e Títulos
class_names = ['Não Cancelado (0)', 'Cancelado (1)']
plt.xticks(np.arange(len(class_names)) + 0.5, class_names, rotation=45, ha='right')
plt.yticks(np.arange(len(class_names)) + 0.5, class_names, rotation=0)

plt.title('Matriz de Confusão - Árvore de Decisão', fontsize=14, fontweight='bold')
plt.xlabel('Valor Previsto')
plt.ylabel('Valor Real')

plt.tight_layout()
plt.show()

print("\n🚨 Compare o canto Inferior Esquerdo (Falsos Negativos) com o modelo Logístico.")
print("O modelo com o menor número nessa célula é o melhor para a segurança operacional!")

### Inferências Finais ###

A Regressão logistica e o Arvore de decisão foram usados para prever se o vôo será cancelado.

- Inferência Principal: O limite de R² em 0.2265 é uma mensagem clara: o erro médio (RMSE) de cerca de 32 minutos não pode ser reduzido com as features que temos.

- Conclusão de Negócio: O modelo atual não se mostrou eficaz para prever o tempo de atraso, possuindo limitações para que sua implementação seja feita. Para que a previsão de atraso seja útil (ex: R² > 0.60 e RMSE < 15 min), Serão necessarios novos dados de dinâmica operacional na base, como:Tráfego/Volume de Voos. Dados de Chegada do Avião Anterior (o famoso atraso em cascata). Condições Meteorológicas 